In [26]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import yfinance as yf

In [28]:
stock_symbol="BPCL.NS"
# import seaborn as sns

start_date='2020-06-01'
end_date='2022-06-25'
df =yf.download(stock_symbol, str(start_date), str(end_date))
# df =yf.download(stock_symbol, period='2y')
# df.reset_index(inplace=True)
# df=pd.DataFrame({'Date':df1.index, 'Close':df1['Close'],'Open':df1['Open'],'High':df1['High'],'Low':df1['Low'],'Adj Close':df1['Adj Close'],'Volume':df1['Volume']})
# df=df1[['Date','Close','Open','High','Low','Adj Close','Volume']]
# df=df.drop(['Date'],axis=1)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-06-01,344.299988,356.700012,340.100006,346.299988,298.089752,10211563
2020-06-02,348.000000,348.000000,337.299988,341.600006,294.044098,6319721
2020-06-03,344.000000,353.950012,339.200012,349.049988,300.456909,10083891
2020-06-04,342.000000,359.399994,339.000000,357.250000,307.515350,16389838
2020-06-05,356.649994,372.500000,352.200012,369.600006,318.146057,10985054
...,...,...,...,...,...,...
2022-06-20,304.000000,304.500000,294.000000,296.049988,296.049988,3728409
2022-06-21,296.649994,302.450012,293.350006,301.200012,301.200012,2801829
2022-06-22,303.149994,307.399994,296.750000,305.899994,305.899994,6602434


In [3]:
df.reset_index(inplace=True)

In [4]:
train_dates = pd.to_datetime(df['Date'])
cols = list(df)[1:7]
df_for_training = df[cols].astype(float)
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)
df_for_training_scaled

array([[-0.32104197, -0.37793524, -0.56948852, -0.59836116, -1.19505997,
        -0.37056515],
       [-0.64234735, -0.60828268, -0.74587495, -0.78829925, -1.37714715,
        -0.11857962],
       [-0.6790681 , -0.31977782, -0.61530311, -0.2528105 , -0.86379057,
         0.40583318],
       ...,
       [-2.36018373, -2.362115  , -2.36198411, -2.26089382, -1.7035165 ,
        -0.12762572],
       [-2.23854628, -2.29141421, -2.20621481, -2.21512554, -1.6525436 ,
        -0.342888  ],
       [-2.24428387, -2.29255426, -2.16383618, -2.22313513, -1.66146401,
        -0.44541923]])

In [5]:
trainX=[]
trainY=[]
n_fut=1
n_past=14

for i in range(n_past, len(df_for_training_scaled) - n_fut +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_fut - 1:i + n_fut, 3])
trainX, trainY = np.array(trainX), np.array(trainY)
trainY

array([[-0.70820476],
       [-0.8031738 ],
       [-0.24708947],
       [ 0.894829  ],
       [ 0.74608208],
       [ 1.36166602],
       [ 1.05044157],
       [ 1.20948593],
       [ 1.00009618],
       [ 1.21291911],
       [ 1.29987856],
       [ 1.1236704 ],
       [ 0.32959115],
       [ 0.20830479],
       [ 0.29984135],
       [ 0.08816305],
       [ 0.13164347],
       [ 0.11104719],
       [ 0.3444657 ],
       [ 0.23919851],
       [ 0.56186503],
       [ 0.39137791],
       [ 0.37650336],
       [ 0.19571879],
       [ 0.0732885 ],
       [-0.05486283],
       [-0.02282447],
       [ 0.17741175],
       [ 0.12363388],
       [ 0.1156243 ],
       [ 0.1877092 ],
       [ 0.12363388],
       [ 0.18885382],
       [ 0.23690996],
       [ 0.07099994],
       [ 0.05841395],
       [ 0.21173796],
       [ 0.19915127],
       [-0.03083406],
       [-0.10291897],
       [ 0.16940217],
       [ 0.02866484],
       [ 0.5858931 ],
       [ 0.55042296],
       [ 0.2849668 ],
       [ 0

In [6]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 14, 64)            18176     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 30,625
Trainable params: 30,625
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='mse')
history = model.fit(trainX, trainY, epochs=5, batch_size=16, validation_split=0.1, verbose=1)
model.save(stock_symbol+'.h5')
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

In [24]:
n_past = 1
n_future=15  #let us predict past 15 days

predict_period_dates = pd.date_range(list(train_dates)[-n_past], periods=n_future, freq='1d').tolist()
prediction = model.predict(trainX[-n_future:])
predict_period_dates

[Timestamp('2022-06-24 00:00:00', freq='D'),
 Timestamp('2022-06-25 00:00:00', freq='D'),
 Timestamp('2022-06-26 00:00:00', freq='D'),
 Timestamp('2022-06-27 00:00:00', freq='D'),
 Timestamp('2022-06-28 00:00:00', freq='D'),
 Timestamp('2022-06-29 00:00:00', freq='D'),
 Timestamp('2022-06-30 00:00:00', freq='D'),
 Timestamp('2022-07-01 00:00:00', freq='D'),
 Timestamp('2022-07-02 00:00:00', freq='D'),
 Timestamp('2022-07-03 00:00:00', freq='D'),
 Timestamp('2022-07-04 00:00:00', freq='D'),
 Timestamp('2022-07-05 00:00:00', freq='D'),
 Timestamp('2022-07-06 00:00:00', freq='D'),
 Timestamp('2022-07-07 00:00:00', freq='D'),
 Timestamp('2022-07-08 00:00:00', freq='D')]

In [9]:
prediction_copies = np.repeat(prediction, df_for_training.shape[1], axis=-1)
y_pred_future = scaler.inverse_transform(prediction_copies)[:,3]

In [10]:
forecast_dates = []
for time_i in predict_period_dates:
    forecast_dates.append(time_i.date())

In [11]:
df_forecast = pd.DataFrame({'Date':np.array(forecast_dates), 'Close':y_pred_future})
df_forecast['Date']=pd.to_datetime(df_forecast['Date'])
df_forecast.index=df_forecast['Date']
df_forecast

,Date,Close
Date,,
2022-06-03,2022-06-03,334.235687
2022-06-04,2022-06-04,332.774750
2022-06-05,2022-06-05,330.473267
2022-06-06,2022-06-06,329.175110
2022-06-07,2022-06-07,330.860718
2022-06-08,2022-06-08,332.769287
2022-06-09,2022-06-09,332.097321
2022-06-10,2022-06-10,329.489410
2022-06-11,2022-06-11,326.261414


In [12]:
original=df[['Date','Close']]
original['Date']=pd.to_datetime(original['Date'])
original.index=original['Date']
original=original.drop(['Date'],axis=1)
original=original.tail(n_future)
original


C:\Users\NEHA RAJESH\AppData\Local\Temp\ipykernel_12196\3244569456.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original['Date']=pd.to_datetime(original['Date'])


,Close
Date,
2022-06-06,319.350006
2022-06-07,320.250000
2022-06-08,323.899994
2022-06-09,332.850006
2022-06-10,331.000000
2022-06-13,317.950012
2022-06-14,314.299988
2022-06-15,314.000000
2022-06-16,311.649994


In [13]:
df_forecast['Actual']=original['Close']
df_forecast=df_forecast.dropna()
df_forecast=df_forecast.drop(['Date'],axis=1)
df_forecast

,Close,Actual
Date,,
2022-06-06,329.175110,319.350006
2022-06-07,330.860718,320.250000
2022-06-08,332.769287,323.899994
2022-06-09,332.097321,332.850006
2022-06-10,329.489410,331.000000
2022-06-13,319.422333,317.950012
2022-06-14,313.433838,314.299988
2022-06-15,308.253693,314.000000
2022-06-16,307.109589,311.649994


In [14]:
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px

In [15]:
fig = px.line( df_forecast,
                        x=df_forecast.index,
                        y=['Close','Actual'],
                        title="Prediction for next {0} days".format(n_future),
                        labels=dict(variable="Prediction"))
fig.update_yaxes(title_text='Prices in ₹')
fig.update_xaxes(title_text='Day')
fig